In [ ]:
from dotenv import dotenv_values
import pandas as pd
from tqdm.notebook import tqdm

from scival_search import RelatedTopics

env_vars = dotenv_values()
COOKIE = env_vars['COOKIE']

df = pd.read_csv("lookups.csv", dtype={"topic": str})
topics = list(set(df['topic'].tolist()))

In [ ]:
# Retrieve related papers for each topic
all_related_topics = []

for topic in tqdm(topics, desc="Topics"):
    related_topics = RelatedTopics(topic, cookie=COOKIE, show_progress=False)
    results = related_topics.results
    results["Topic"] = topic
    results_dict = results[["Topic Number", "Prominence percentile", "Topic"]].to_dict(orient="records")
    all_related_topics.extend(results_dict)

In [ ]:
df_related_topics = pd.DataFrame(all_related_topics).to_csv("related_topics.csv", index=False)

In [ ]:
# Validate: duplicates and missing topics
import pandas as pd

df_related = pd.read_csv("related_topics.csv", dtype={"Topic": str})
df_todo = pd.read_csv("lookups.csv", dtype={"topic": str})

topics_todo = set(df_todo['topic'].tolist())
topics_done = set(df_related['Topic'].tolist())

missing_topics = topics_todo - topics_done
duplicate_topics = df_related[df_related.duplicated(subset=['Topic', 'Topic Number'], keep=False)]

print(f"Missing topics: {len(missing_topics)}")
print(f"Duplicate topics: {len(duplicate_topics)}")